## JSON creation

In [4]:
import os
from src.api.json_creation import JSONCreation
from src.db_connector import DbConnector

db_connector = DbConnector()

json_creator = JSONCreation(os.getenv("S3_CF_BUCKET"), os.getenv("CF_DISTRIBUTION_ID"), db_connector, "v1")
## for testing
#json_creator = JSONCreation(os.getenv("S3_CF_BUCKET"), os.getenv("CF_DISTRIBUTION_ID"), db_connector, "test")

#json_creator.create_all_jsons()

df = json_creator.get_all_data()
#json_creator.create_landingpage_json(df)
#json_creator.create_chain_details_jsons(df)
#json_creator.create_metric_details_jsons(df)
json_creator.create_master_json()

... uploaded to v1/master


## L2Beat

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_l2beat import AdapterL2Beat

adapter_params = {
}
load_params = {
    'origin_keys' : None,
    #'origin_keys' : ['optimism','arbitrum'] # see all options in adapter_mapping.py 
}

# initialize adapter
ad = AdapterL2Beat(adapter_params, db_connector)
# extract
df= ad.extract(load_params)
# load
ad.load(df)

## DefiLlama

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_defillama import AdapterDefiLlama

adapter_params = {
}
load_params = {
    'origin_keys' : None,
    #'origin_keys' : ['ethereum'] # see all options in adapter_mapping.py
}

# initialize adapter
ad = AdapterDefiLlama(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

## Coingecko

run for projects / chains

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_coingecko import AdapterCoingecko

adapter_params = {
}
load_params = {
    'load_type' : 'project',
    'metric_keys' : ['price', 'volume', 'market_cap'],
    'origin_keys' : None,
    #'origin_keys' : ['aptos'], # see all options in adapter_mapping.py
    'days' : 'auto', # auto, max, or a number (as string)
    'vs_currencies' : ['usd', 'eth']
}

# initialize adapter
ad = AdapterCoingecko(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

and for imx tokens

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_coingecko import AdapterCoingecko

adapter_params = {
}
load_params = {
    'load_type' : 'imx_tokens'
}

# initialize adapter
ad = AdapterCoingecko(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

## Dune

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_dune import AdapterDune

adapter_params = {
    'api_key' : os.getenv("DUNE_API")
}

load_params = {
    'query_names' : ['fundamentals'], ## fundamentals, waa, stables_mcap
    'days' : 2000,
    #'query_names' : None,
    #'days' : 'auto',
}

# initialize adapter
ad = AdapterDune(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# upload
ad.load(df)

In [ ]:
ad.load(df)

## Flipside
sometimes some Flipside queries just get stuck -- gotta retrigger them

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_flipside import AdapterFlipside

adapter_params = {
    'api_key' : os.getenv("FLIPSIDE_API")
}
load_params = {
    'origin_keys' : ['zksync_era'],
    'metric_keys' : ['stables_mcap'],
    'days' : 'auto',
    # 'origin_keys' : None,
    # 'metric_keys' : None,
    # 'days' : 'auto',
}

# initialize adapter
ad = AdapterFlipside(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

In [ ]:
## testing the new flipside api

from flipside import Flipside
flipside = Flipside("74c43ebc-3291-4953-8aeb-65640da7c852", "https://api-v2.flipsidecrypto.xyz")

sql = """
 select 
    BLOCK_NUMBER, BLOCK_TIMESTAMP, BLOCK_HASH, TX_HASH, NONCE, POSITION, ORIGIN_FUNCTION_SIGNATURE, FROM_ADDRESS, TO_ADDRESS, ETH_VALUE, TX_FEE, GAS_PRICE, GAS_LIMIT, 
    GAS_USED, L1_GAS_PRICE, L1_GAS_USED, L1_FEE_SCALAR, L1_SUBMISSION_BATCH_INDEX, L1_SUBMISSION_TX_HASH, L1_STATE_ROOT_BATCH_INDEX, 
    L1_STATE_ROOT_TX_HASH, CUMULATIVE_GAS_USED, INPUT_DATA, STATUS
from optimism.core.fact_transactions
where block_number >= 104130000
and block_number < 104135000
order by block_number asc
"""

# Run the query against Flipside's query engine and await the results
query_result_set = flipside.query(sql)

## Zettablock

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_zettablock import AdapterZettablock

adapter_params = {
    'api_key' : os.getenv("ZETTABLOCK_API")
}
load_params = {
    #'origin_keys' : ['zksync_era', 'polygon_zkevm'],
    #'metric_keys' : ['txcount', ''],
    'days' : 10,
    'origin_keys' : None,
    'metric_keys' : None,
    # 'days' : 'auto',
}

# initialize adapter
ad = AdapterZettablock(adapter_params, db_connector)
# extract
df = ad.extract(load_params)
# load
ad.load(df)

## ZettaBlock raw data load

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_zettablock import AdapterZettaBlockRaw

adapter_params = {
    'api_key' : os.getenv("ZETTABLOCK_API")
}

load_params = {
    #'keys' : ['polygon_zkevm_tx', 'zksync_era_tx'],
    'keys' : ['zksync_era_tx'],
    #'block_start' : 'auto', ## 'auto' or a block number as int
    'block_start' : 9137631, ## 'auto' or a block number as int
}

# initialize adapter
ad = AdapterZettaBlockRaw(adapter_params, db_connector)
# extract
ad.extract_raw(load_params, if_exists = 'ignore')

## Chainbase raw

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_chainbase import AdapterChainbaseRaw

adapter_params = {
    'api_key' : os.getenv("CHAINBASE_API")
}

load_params = {
    'keys' : ['arbitrum_tx'],
    #'block_start' : 'auto', ## 'auto' or a block number as int
    'block_start' : 64900000, ## until 65,570,000
}

# initialize adapter
ad = AdapterChainbaseRaw(adapter_params, db_connector)
# extract
ad.extract_raw(load_params)

## IMX raw data load

In [ ]:
## implement orchestration?

from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_imx import AdapterRawImx

adapter_params = {
    'load_types' : ['withdrawals', 'deposits', 'trades', 'orders_filled', 'transfers', 'mints'],
    'forced_refresh' : 'no',

    #'load_types' : ['orders_filled'],
    #'forced_refresh' : '2023-04-01T00:00:00.000000Z',
}

# initialize adapter
ad = AdapterRawImx(adapter_params, db_connector)
# extract raw (and load raw in case of IMX)
df_raw = ad.extract_raw()

## Flipside raw data load

In [ ]:
import os
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_raw_flipside import AdapterFlipsideRaw

adapter_params = {
    'api_key' : os.getenv("FLIPSIDE_API")
}

load_params = {
    'keys' : ['arbitrum_tx', 'optimism_tx'],
    'block_start' : 'auto',
}

# initialize adapter
ad = AdapterFlipsideRaw(adapter_params, db_connector)
# extract
df = ad.extract_raw(load_params)

## SQL aggregation

In [ ]:
## Loads currently IMX txcount, daa, fees_paid
## also loads user_base_weekly

from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_sql import AdapterSQL

adapter_params = {
}

load_params = {
    'load_type' : 'metrics', ## usd_to_eth or metrics
    
    #'days' : 'auto', ## days as int our 'auto
    'origin_keys' : None, ## origin_keys as list or None
    #'metric_keys' : None, ## metric_keys as list or None

    'days' : '28', ## days as int our 'auto
    #'origin_keys' : ['imx'], ## origin_keys as list or None
    'metric_keys' : ['user_base_weekly'], ## metric_keys as list or None
}

# initialize adapter
ad = AdapterSQL(adapter_params, db_connector)

# extract
df = ad.extract(load_params)
# # load
ad.load(df)

In [ ]:
df.origin_key.unique()

In [ ]:
from src.db_connector import DbConnector
db_connector = DbConnector()
from src.adapters.adapter_sql import AdapterSQL

adapter_params = {
}

load_params = {
    'load_type' : 'usd_to_eth', ## usd_to_eth or metrics
    'days' : 2000, ## days as int
    'origin_keys' : None, ## origin_keys as list or None
    'metric_keys' : None, ## metric_keys as list or None
}

# initialize adapter
ad = AdapterSQL(adapter_params, db_connector)

# extract
df = ad.extract(load_params)
# # load
ad.load(df)

### Blockspace logic
- for each chain, aggregate the daily contracts usage


In [1]:
from src.db_connector import DbConnector
db_connector = DbConnector()

chain_list = ['arbitrum', 'zksync_era', 'polygon_zkevm']

In [2]:
for chain in chain_list:
    days =  db_connector.get_last_blockspace_date(chain)

    # ## aggregate contract data
    # print(f"aggregating contract data for {chain} and last {days} days...")
    # df = db_connector.get_blockspace_contracts(chain, days)
    # df.set_index(['address', 'date', 'origin_key'], inplace=True)

    # print(f"upserting contract data for {chain}. Total rows: {df.shape[0]}...")
    # db_connector.upsert_table('blockspace_fact_contract_level', df)

    # ## aggregate native transfers
    # print(f"aggregating native_transfers for {chain} and last {days} days...")
    # df = db_connector.get_blockspace_native_transfers(chain, days)
    # df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    # print(f"upserting native_transfers for {chain}. Total rows: {df.shape[0]}...")
    # db_connector.upsert_table('blockspace_fact_sub_category_level', df)

    ## aggregate contract deployments
    print(f"aggregating smart_contract_deployments for {chain} and last {days} days...")
    df = db_connector.get_blockspace_contract_deplyments(chain, days)
    df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    print(f"upserting smart_contract_deployments for {chain}. Total rows: {df.shape[0]}...")
    db_connector.upsert_table('blockspace_fact_sub_category_level', df)

    # # ALL below needs to be retriggerd when mapping changes (e.g. new addresses got labeled or new categories added etc.)
    # ## aggregate by sub categories
    # print(f"aggregating sub categories for {chain} and last {days} days...")
    # df = db_connector.get_blockspace_sub_categories(chain, days)
    # df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    # print(f"upserting sub categories for {chain}. Total rows: {df.shape[0]}...")
    # db_connector.upsert_table('blockspace_fact_sub_category_level', df)

    # ## determine unlabeled usage
    # print(f"aggregating unlabeled usage for {chain} and last {days} days...")
    # df = db_connector.get_blockspace_unlabeled(chain, days)
    # df.set_index(['date', 'sub_category_key' ,'origin_key'], inplace=True)

    # print(f"upserting unlabeled usage for {chain}. Total rows: {df.shape[0]}...")
    # db_connector.upsert_table('blockspace_fact_sub_category_level', df)

aggregating smart_contract_deployments for arbitrum and last 3000 days...
upserting smart_contract_deployments for arbitrum. Total rows: 464...
aggregating smart_contract_deployments for zksync_era and last 3000 days...
upserting smart_contract_deployments for zksync_era. Total rows: 150...
aggregating smart_contract_deployments for polygon_zkevm and last 3000 days...
upserting smart_contract_deployments for polygon_zkevm. Total rows: 0...


In [1]:
from src.db_connector import DbConnector
db_connector = DbConnector()

In [4]:
df = db_connector.get_top_contracts_by_category('main_category', 'defi', 'arbitrum', 'gas', 7)

In [5]:
df

,address,origin_key,contract_name,project_name,sub_category_key,sub_category_name,main_category_key,main_category_name,gas_fees_eth,gas_fees_usd,txcount,daa
0,"[b'\xb4', b'1', b'^', b'\x87', b'=', b'\xbc', ...",arbitrum,LBRouter,trader_joe_v2_1,DEX,Decentralized Exchanges,defi,DeFi,3.155552,5955.160109,24064.0,16007.0
1,"[b'\xde', b'\xf1', b'\xc0', b'\xde', b'\xd9', ...",arbitrum,ExchangeProxy,zeroex,DEX,Decentralized Exchanges,defi,DeFi,3.028383,5710.158131,16493.0,9985.0
2,"[b'\x1b', b'\x02', b'\xda', b'\x8c', b'\xb0', ...",arbitrum,UniswapV2Router02,sushi,DEX,Decentralized Exchanges,defi,DeFi,2.736958,5161.087733,24751.0,11936.0
3,"[b'\xdd', b'\x94', b'\x01', b'\x8f', b'T', b'\...",arbitrum,OdosRouter,odos_v1,DEX,Decentralized Exchanges,defi,DeFi,2.178278,4108.596571,10607.0,6131.0
4,"[b'\xf4', b'\xb1', b'H', b'm', b'\xd7', b'M', ...",arbitrum,LendingPool,radiant_capital,lending,Lending,defi,DeFi,1.893687,3571.395943,11991.0,5697.0
5,"[b'U', b'C', b'U', b'\r', b'e', b'\x81', b'<',...",arbitrum,SwapRouter,camelot,DEX,Decentralized Exchanges,defi,DeFi,1.483005,2798.396560,10238.0,6623.0
6,"[b'\xc8', b's', b'\xfe', b'\xcb', b'\xd3', b'T...",arbitrum,CamelotRouter,camelot,DEX,Decentralized Exchanges,defi,DeFi,1.071587,2022.078110,8719.0,4839.0
7,"[b'\x11', b'\x11', b'\x11', b'\x12', b'T', b'\...",arbitrum,AggregationRouterV4,oneinch,DEX,Decentralized Exchanges,defi,DeFi,0.941212,1775.969923,7150.0,5061.0
8,"[b'\xb4', b'Z', b'-', b'\xda', b'\x99', b'l', ...",arbitrum,TransitSwapRouterV4,transit_finance,DEX,Decentralized Exchanges,defi,DeFi,0.347513,655.850070,1969.0,1292.0
9,"[b'\xb9', b']', b'\xb5', b'\xb1', b'g', b'\xd7...",arbitrum,RewardRouterV2,gmx,staking,Staking,defi,DeFi,0.326638,616.010679,1789.0,1480.0


## S3

In [ ]:
## connect to s3 bucket and output list of files
import pandas as pd
import boto3
import os

s3 = boto3.resource(
    service_name='s3',
    region_name='us-east-1',
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY")
)

bucket = s3.Bucket('gtp-longterm')

## get list of files in bucket with last modified date
files = []
for obj in bucket.objects.all():
    files.append([obj.key, obj.last_modified])

df = pd.DataFrame(files, columns=['key', 'last_modified'])

## filter out files where key starts with 'imx'
df = df[~df.key.str.startswith('imx')]

df['chain'] = df.key.str.split('/').str[0]

## create new column block_range that extracts the string between 'tx_' and '.parquet' in the key column using lambda function
df['block_range'] = df.key.apply(lambda x: x[x.find('tx_')+3:x.find('.parquet')])
df['block_start'] = df.block_range.str.split('-').str[0].astype(int)
df['block_end'] = df.block_range.str.split('-').str[1].astype(int)

## sort by block_start
df.sort_values(by='block_start', inplace=True, ascending=True)

In [ ]:
df

### Arbitrum

In [ ]:
df_arbitrum = df[df.chain == 'arbitrum']

In [ ]:
df_arbitrum

In [ ]:
## load first file in df_arbitrum into df
df_arbitrum_flipside = pd.read_parquet(f"s3://gtp-longterm/{df_arbitrum.key.iloc[0]}")
df_arbitrum_flipside.sort_values(by='ETH_VALUE', inplace=True, ascending=False)
df_arbitrum_flipside.head(10)

In [ ]:
## load first file in df_arbitrum with block_start >= 96528665 into df
df_arbitrum_chainbase = pd.read_parquet(f"s3://gtp-longterm/{df_arbitrum[df_arbitrum.block_start >= 96528665].key.iloc[10]}")
df_arbitrum_chainbase.head()

### Optimism

In [ ]:
df_optimism = df[df.chain == 'optimism']

In [ ]:
## load first file in df_arbitrum into df
df_optimism_flipside = pd.read_parquet(f"s3://gtp-longterm/{df_optimism.key.iloc[0]}")
df_optimism_flipside.head()

In [ ]:
# df_optimism_flipside[['TX_HASH', 'ETH_VALUE', 'TX_FEE', 'GAS_PRICE', 'GAS_LIMIT', 'GAS_USED',
#        'L1_GAS_PRICE', 'L1_GAS_USED', 'L1_FEE_SCALAR',  'CUMULATIVE_GAS_USED' ]]

## filter df_optimism_flipside where TX_HASH = 0xA5E0DACD8D29020C04139F8506C01CBA8B6E561CE567DF8DA35857722232F559 and select columns 'TX_HASH', 'ETH_VALUE', 'TX_FEE', 'GAS_PRICE', 'GAS_LIMIT', 'GAS_USED','L1_GAS_PRICE', 'L1_GAS_USED', 'L1_FEE_SCALAR',  'CUMULATIVE_GAS_USED' 

df_optimism_flipside[df_optimism_flipside.TX_HASH == '0xA5E0DACD8D29020C04139F8506C01CBA8B6E561CE567DF8DA35857722232F559'.lower()][['TX_HASH', 'ETH_VALUE', 'TX_FEE', 'GAS_PRICE', 'GAS_LIMIT', 'GAS_USED','L1_GAS_PRICE', 'L1_GAS_USED', 'L1_FEE_SCALAR',  'CUMULATIVE_GAS_USED' ]]

In [ ]:
df_optimism_flipside['GAS_PRICE'] / 1e9

In [ ]:
df_optimism_flipside.columns

In [ ]:
## load first file in df_arbitrum with block_start >= 96528665 into df
df_optimism_chainbase = pd.read_parquet(f"s3://gtp-longterm/{df_optimism[df_optimism.block_start >= 103428989].key.iloc[0]}")
df_optimism_chainbase.head()

### Polygon zkEVM

In [ ]:
df_polygon = df[df.chain == 'polygon_zkevm']

In [ ]:
## load first file in df_arbitrum into df
df_polygon_zb = pd.read_parquet(f"s3://gtp-longterm/{df_polygon.key.iloc[0]}")
df_polygon_zb.head()

### zkSync Era

In [ ]:
df_zksync = df[df.chain == 'zksync_era']

In [ ]:
## load first file in df_arbitrum into df
df_zksync_zb = pd.read_parquet(f"s3://gtp-longterm/{df_zksync.key.iloc[0]}")
df_zksync_zb.head()

In [ ]:
# ## rename files in S3 bucket that contain 'block_'
# for index, row in df[df.key.str.contains('block_')].iterrows():
#     print(row['key'])
#     old_key = row['key']
#     new_key = old_key.replace('block_', '')
#     print(new_key)
#     s3.Object('gtp-longterm', new_key).copy_from(CopySource='gtp-longterm/'+old_key)
#     #s3.Object('gtp-longterm', old_key).delete()


In [ ]:
df_arbitrum[df_arbitrum.block_start >= 96528665].sort_values(by='last_modified', ascending=True)

In [ ]:
main_zksync()

## indexed.xyz

In [ ]:
import boto3

aws_access_key_id = "43c31ff797ec2387177cabab6d18f15a"
aws_secret_access_key = "afb354f05026f2512557922974e9dd2fdb21e5c2f5cbf929b35f0645fb284cf7"
bucket_name = 'indexed-xyz'

s3 = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
        )

In [ ]:
## list all files in bucket
for obj in s3.list_objects(Bucket=bucket_name)['Contents']:
    print(obj['Key'])